In [1]:
import pandas as pd
import numpy as np
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm

In [2]:
def calc_panel_robust_vcov(model, n_individuals): #n_individuals means how many members/individuals in the data, can check how many different rows in one time period
    """
    Calculate panel-robust variance-covariance matrix based on equation (11-3)
    """
    X = model.model.exog
    n = X.shape[0]
    k = X.shape[1]
    
    # Get residuals
    resid = np.array(model.resid)
    
    # Calculate (X'X)^(-1)
    XX_inv = np.linalg.inv(np.dot(X.T, X) / n)
    
    # Initialize middle matrix
    S = np.zeros((k, k))
    
    # Calculate the middle matrix using individual clusters
    obs_per_indiv = n // n_individuals
    for i in range(n_individuals):
        # Get slice for individual i
        start_idx = i * obs_per_indiv
        end_idx = (i + 1) * obs_per_indiv
        
        # Get X_i and residuals for individual i
        X_i = X[start_idx:end_idx]
        resid_i = resid[start_idx:end_idx]
        
        # Calculate w_i * w_i' for individual i
        w_i = resid_i.reshape(-1, 1)
        w_i_w_i = np.dot(w_i, w_i.T)
        
        # Calculate X_i' * w_i * w_i' * X_i
        middle_term = np.dot(np.dot(X_i.T, w_i_w_i), X_i)
        
        # Add to S matrix
        S += middle_term
    
    # Calculate robust variance-covariance matrix
    S = S / n
    vcov = np.dot(np.dot(XX_inv, S), XX_inv) / n
    
    return vcov

In [3]:
df = pd.read_csv('example data.csv')
n_individuals = int(df["T"].value_counts()[1]) # Number of individuals in your panel

In [4]:
# Create Exp squared term
df['EXP2'] = df['EXP'] ** 2

# Prepare independent variables
X = df[['EXP', 'EXP2', 'WKS', 'OCC', 'IND', 'SOUTH', 
        'SMSA', 'MS', 'UNION', 'ED', 'FEM', 'BLK']]
X = sm.add_constant(X)

# Dependent variable
y = df['LWAGE']

# Run OLS regression
model = OLS(y, X).fit()

# Get different standard errors
ols_se = model.bse
white_se = model.get_robustcov_results(cov_type='HC0').bse

# Calculate panel-robust standard errors
panel_vcov = calc_panel_robust_vcov(model, n_individuals)
panel_se = np.sqrt(np.diag(panel_vcov))

# Create results DataFrame
results = pd.DataFrame({
    'Variable': X.columns,
    'Estimated_Coefficient': model.params,
    'OLS_Standard_Error': ols_se,
    'Panel_Robust_SE': panel_se,
    'White_Hetero_SE': white_se,
})

In [5]:
# Round numeric columns to 5 decimal places
numeric_cols = ['Estimated_Coefficient', 'OLS_Standard_Error', 
               'Panel_Robust_SE', 'White_Hetero_SE']
results[numeric_cols] = results[numeric_cols].round(5)

In [8]:
print(results)

      Variable  Estimated_Coefficient  OLS_Standard_Error  Panel_Robust_SE  \
const    const                5.25112             0.07129          0.12326   
EXP        EXP                0.04010             0.00216          0.00407   
EXP2      EXP2               -0.00067             0.00005          0.00009   
WKS        WKS                0.00422             0.00108          0.00154   
OCC        OCC               -0.14001             0.01466          0.02718   
IND        IND                0.04679             0.01179          0.02361   
SOUTH    SOUTH               -0.05564             0.01253          0.02610   
SMSA      SMSA                0.15167             0.01207          0.02405   
MS          MS                0.04845             0.02057          0.04085   
UNION    UNION                0.09263             0.01280          0.02362   
ED          ED                0.05670             0.00261          0.00555   
FEM        FEM               -0.36779             0.02510       

In [69]:
# def calc_panel_robust_vcov(model, n_individuals):
#     """
#     Calculate panel-robust variance-covariance matrix based on equation (11-3)
#     """
#     X = model.model.exog
#     n = X.shape[0]
#     k = X.shape[1]
    
#     # Get residuals
#     resid = np.array(model.resid)
    
#     # Calculate (X'X)^(-1)
#     XX_inv = np.linalg.inv(np.dot(X.T, X) / n)
    
#     # Initialize middle matrix
#     S = np.zeros((k, k))
    
#     # Calculate the middle matrix using individual clusters
#     obs_per_indiv = n // n_individuals
#     for i in range(n_individuals):
#         # Get slice for individual i
#         start_idx = i * obs_per_indiv
#         end_idx = (i + 1) * obs_per_indiv
        
#         # Get X_i and residuals for individual i
#         X_i = X[start_idx:end_idx]
#         resid_i = resid[start_idx:end_idx]
        
#         # Calculate w_i * w_i' for individual i
#         w_i = resid_i.reshape(-1, 1)
#         w_i_w_i = np.dot(w_i, w_i.T)
        
#         # Calculate X_i' * w_i * w_i' * X_i
#         middle_term = np.dot(np.dot(X_i.T, w_i_w_i), X_i)
        
#         # Add to S matrix
#         S += middle_term
    
#     # Calculate robust variance-covariance matrix
#     S = S / n
#     vcov = np.dot(np.dot(XX_inv, S), XX_inv) / n
    
#     return vcov

# def run_regression(df, n_individuals):
#     # Create Exp squared term
#     df['EXP2'] = df['EXP'] ** 2
    
#     # Prepare independent variables
#     X = df[['EXP', 'EXP2', 'WKS', 'OCC', 'IND', 'SOUTH', 
#             'SMSA', 'MS', 'UNION', 'ED', 'FEM', 'BLK']]
#     X = sm.add_constant(X)
    
#     # Dependent variable
#     y = df['LWAGE']
    
#     # Run OLS regression
#     model = OLS(y, X).fit()
    
#     # Get different standard errors
#     ols_se = model.bse
#     white_se = model.get_robustcov_results(cov_type='HC0').bse
    
#     # Calculate panel-robust standard errors
#     panel_vcov = calc_panel_robust_vcov(model, n_individuals)
#     panel_se = np.sqrt(np.diag(panel_vcov))
    
#     # Create results DataFrame
#     results = pd.DataFrame({
#         'Variable': X.columns,
#         'Estimated_Coefficient': model.params,
#         'OLS_Standard_Error': ols_se,
#         'Panel_Robust_SE': panel_se,
#         'White_Hetero_SE': white_se,
#     })
    
#     return results, model

# def format_results(results):
#     # Round numeric columns to 5 decimal places
#     numeric_cols = ['Estimated_Coefficient', 'OLS_Standard_Error', 
#                    'Panel_Robust_SE', 'White_Hetero_SE']
#     results[numeric_cols] = results[numeric_cols].round(5)
    
#     return results

# # Example usage:
# # Assuming your data is in a DataFrame called 'df' and you know the number of individuals

# df = pd.read_csv('example data.csv')
# n_individuals = int(df["T"].value_counts()[1]) # Number of individuals in your panel
# results, model = run_regression(df, n_individuals)
# formatted_results = format_results(results)
# print(formatted_results)

      Variable  Estimated_Coefficient  OLS_Standard_Error  Panel_Robust_SE  \
const    const                5.25112             0.07129          0.12326   
EXP        EXP                0.04010             0.00216          0.00407   
EXP2      EXP2               -0.00067             0.00005          0.00009   
WKS        WKS                0.00422             0.00108          0.00154   
OCC        OCC               -0.14001             0.01466          0.02718   
IND        IND                0.04679             0.01179          0.02361   
SOUTH    SOUTH               -0.05564             0.01253          0.02610   
SMSA      SMSA                0.15167             0.01207          0.02405   
MS          MS                0.04845             0.02057          0.04085   
UNION    UNION                0.09263             0.01280          0.02362   
ED          ED                0.05670             0.00261          0.00555   
FEM        FEM               -0.36779             0.02510       

ED
12    1498
16     637
17     504
14     343
10     231
11     210
13     182
8      175
9      161
15      84
7       77
6       28
5       21
4       14
Name: count, dtype: int64